<a href="https://colab.research.google.com/github/clarkvd/NLPFinalProject/blob/main/NLP_Final_Project_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important** Spotify Data is using binary classification for BERT (eg. positive and negative)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import pandas as pd
import nltk
import gensim

from sklearn.datasets import fetch_openml as fetch_mldata
from sklearn.model_selection import cross_val_score

from numpy import mean
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics

In [ ]:
spotifyData = fetch_mldata(name='150K-Lyrics-Labeled-with-Spotify-Valence', version=1)
sdf = spotifyData.data
spotify_x = sdf["seq"]
spotify_y = sdf["label"]
print(spotify_x[1])
print(spotify_y[1])   

The drinks go down and smoke goes up, I feel myself, got to let go
My cares get lost up in that crowd that go up, up and away yo

Slow down the lights, eyes open wide
We live till we die, live till we die, live till we die

Ain't kill my vibe, don't blow my high, don't doubt that he from the band though
I'm listening to this song, now I'm up up and away yo

Slow down the lights, eyes open wide
We live till we die, live till we die, live till we die
Slow down the lights, eyes open wide
We live till we die, live till we die, live till we die
Slow down the lights, eyes open wide
We live till we die, live till we die, live till we die

Nights like this, I go all out, up so high, I can't come down
Let me live just for right now
Yeah, yeah, bite me, I'm so gone

Happier than a motherfucker, I can't feel my fa-ace
No squares in my circle, get up out my way
Happier than a motherfucker, I can't feel my fa-ace
Who you tryna dance? Get up out my way
Happier than a motherfucker, I can't feel my fa

In [ ]:
# print(sdf.shape)  
valence = [] 
bertvalencetest = []
count = len(sdf) 
vn = 0 
sn = 0 
n = 0 
sp = 0 
vp = 0 
# Bin valences 
for val in spotify_y: 
  if val<0.5:
    bertvalencetest.append(0)
  if val>=0.5 and val<1:
    bertvalencetest.append(1)
    
sdf['bertvalence'] = bertvalencetest

In [ ]:
sdf = spotifyData.data

## We are using a 80/20 split 
spotify_training_data = sdf.sample(frac=0.8, random_state=25)
spotify_testing_data = sdf.drop(spotify_training_data.index)
print(f"No. of spotify training examples: {spotify_training_data.shape[0]}")
print(f"No. of spotify testing examples: {spotify_testing_data.shape[0]}") 

No. of spotify training examples: 126682
No. of spotify testing examples: 31671


Install Transformers and Datasets

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 16.2 MB/s 
     |████████████████████████████████| 7.6 MB 70.9 MB/s 
     |████████████████████████████████| 182 kB 83.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 13.9 MB/s 
     |████████████████████████████████| 212 kB 99.9 MB/s 
     |████████████████████████████████| 132 kB 109.1 MB/s 
     |████████████████████████████████| 127 kB 102.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import Dataset
import pandas as pd
from datasets import load_dataset

trai = []
tes = []

df = pd.DataFrame({"lyrics_tokenized": spotify_training_data["seq"], "labels":spotify_training_data['bertvalence']})
df2 = pd.DataFrame({"lyrics_tokenized": spotify_testing_data["seq"], "labels":spotify_testing_data['bertvalence']})

df.to_csv(r'/content/drive/MyDrive/train.csv',index=False)
df2.to_csv(r'/content/drive/MyDrive/test.csv',index=False)

print(df.head())
dataset = load_dataset('csv',data_files={'train': '/content/drive/MyDrive/train.csv', 
                                           'test': '/content/drive/MyDrive/test.csv'})

print(dataset)

                                         lyrics_tokenized  labels
75388   Hey, Bungalow Bill\nWhat did you kill\nBungalo...       1
23762   Uncertainty follows me like a shade and it fog...       1
85212   [Dialogue borrowed from a "Rocky" movie]\nThis...       0
44119   This is the death\nThe death of rock\nYou may ...       0
143317  Welcome to the system\nHere's the situation\nI...       1


Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8e3b996651ceaa6d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['lyrics_tokenized', 'labels'],
        num_rows: 126682
    })
    test: Dataset({
        features: ['lyrics_tokenized', 'labels'],
        num_rows: 31671
    })
})


In [ ]:
#RUN THIS ************
small_train_dataset = dataset["train"].shuffle(seed=42).select([i for i in list(range(6000))])
small_test_dataset = dataset["test"].shuffle(seed=42).select([i for i in list(range(600))])

In [ ]:
#RUN THIS ************
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#RUN THIS ************
def preprocess_function(examples):
  return tokenizer(examples["lyrics_tokenized"], truncation=True, padding=True)

In [ ]:
#RUN THIS ************
tokenized_small_train = small_train_dataset.map(preprocess_function)
tokenized_small_test = small_test_dataset.map(preprocess_function)
tokenized_small_train[0]

  0%|          | 0/6000 [00:00<?, ?ex/s]

  0%|          | 0/600 [00:00<?, ?ex/s]

{'lyrics_tokenized': "Put your swords away I'm already down\nThe higher we both climb the further we fall\nNow were under looking up into the floor\nAnd burn each other though we're dying for more\nWe're stuck on this dysfunction We love it to our death\nWe celebrate destruction by lighting up this bed....\n\nYou can take your love away but don't you ever leave alone\nBurn the ghosts we've carried home\nCome on touch me show me your imperfect too\nThese broken lights they shine on us tonight\n\nWe don't talk much when there's so much to say\nWe don't listen but yet we try anyway\nMy stomach sick bitter and open \nReveries of helplessness have left me all but hopin'\n\nWe strip each others beauty and we push it to the end\nWe face each other smiling and leave it all unsaid\n\nWe kiss till we bleed \nWe Feel So much its killing us\nWe fight till were free\nAnd he threatens to leave but won't let me\nWe cry till we see \nInfatuations Break Us down\nWere healing at three\nWe dance inside t

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [ ]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels,average="weighted")["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_small_train,
    eval_dataset=tokenized_small_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: lyrics_tokenized. If lyrics_tokenized are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6000
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5625
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__c

Step,Training Loss
500,0.641500
1000,0.553100
1500,0.396800
2000,0.216700
2500,0.159100
3000,0.111700
3500,0.075300


Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-500
Configuration saved in /content/drive/MyDrive/results/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/results/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-1000
Configuration saved in /content/drive/MyDrive/results/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/results/checkpoint-1500
Configuration saved in /content/drive/MyDri

KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()

Genre Classification BERT

In [ ]:
genreData =  fetch_mldata(name='Music-Dataset--1950-to-2019', version=1)
gdf = genreData.data
x = gdf["lyrics"]
y = gdf["valence"]

In [ ]:
# print(gdf.head())
print(sdf.head())

In [ ]:

bertvalencetestg = []
for val in gdf['valence']: 
  if val<0.5:
    bertvalencetestg.append(0)
  if val>=0.5 and val<=1:
    bertvalencetestg.append(1)
print(len(bertvalencetestg))
print(gdf.shape)
gdf['bert'] = bertvalencetestg

# for genre in genre_training_data["genre"]:
#   if genre == 'rock': 
#     trai.append(0)
#   elif genre == 'country': 
#     trai.append(1)
#   elif genre == 'blues': 
#     trai.append(2)
#   elif genre == 'jazz': 
#     trai.append(3)
#   elif genre == 'reggae': 
#     trai.append(4) 
#   elif genre == 'hip hop': 
#     trai.append(5)
# for genre in genre_testing_data["genre"]:
#   if genre == 'rock': 
#     tes.append(0)
#   elif genre == 'country': 
#     tes.append(1)
#   elif genre == 'blues': 
#     tes.append(2)
#   elif genre == 'jazz': 
#     tes.append(3)
#   elif genre == 'reggae': 
#     tes.append(4) 
#   elif genre == 'hip hop': 
#     tes.append(5)
  
genre_training_data = gdf.sample(frac=0.8, random_state=25)
genre_testing_data = gdf.drop(genre_training_data.index)

df = pd.DataFrame({"text": genre_training_data["lyrics"], "labels":genre_training_data['bert']})
df2 = pd.DataFrame({"text": genre_testing_data["lyrics"], "labels":genre_testing_data['bert']})

df.to_csv(r'/content/drive/MyDrive/train.csv',index=False)
df2.to_csv(r'/content/drive/MyDrive/test.csv',index=False)

print(df.head())
dataset = load_dataset('csv',data_files={'train': '/content/drive/MyDrive/train.csv', 
                                           'test': '/content/drive/MyDrive/test.csv'})

print(dataset)

In [ ]:
# gdf.drop(gdf.loc[gdf['genre'] == 'pop'].index, inplace=True)
# print(gdf.shape)

In [ ]:
# genre_training_data = gdf.sample(frac=0.8, random_state=25)
# genre_testing_data = gdf.drop(genre_training_data.index)

# print(f"No. of genre training examples: {genre_training_data.shape[0]}")
# print(f"No. of genre testing examples: {genre_testing_data.shape[0]}")  

In [ ]:
small_train_dataset2 = dataset["train"].shuffle(seed=42).select([i for i in list(range(10000))])
small_test_dataset2 = dataset["test"].shuffle(seed=42).select([i for i in list(range(1000))])

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True, padding=True)

In [ ]:
tokenized_small_train2 = small_train_dataset2.map(preprocess_function)
tokenized_small_test2 = small_test_dataset2.map(preprocess_function)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
genre_model = model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/lab8/results2",
    learning_rate=2e-5,
    per_device_train_batch_size= 16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=genre_model,
    args=training_args,
    train_dataset=tokenized_small_train2,
    eval_dataset=tokenized_small_test2,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

RNN

In [ ]:
nltk.download('punkt');
def tokenize(column):
    tokens = nltk.word_tokenize(column)
    return [w for w in tokens if w.isalpha()]  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sdf['lyrics_tokenized'] = sdf.apply(lambda x: tokenize(x['seq']), axis=1)
sdf[['lyrics_tokenized']].head() 

,lyrics_tokenized
0,"[No, no, I, ai, ever, trapped, out, the, bando..."
1,"[The, drinks, go, down, and, smoke, goes, up, ..."
2,"[She, do, live, on, planet, Earth, no, more, S..."
3,"[Trippin, off, that, Grigio, mobbin, lights, l..."
4,"[I, see, a, midnight, panther, so, gallant, an..."


In [ ]:
## Can we make sad Country songs? Can we make happy hip-hop songs?

justin_bieber = sdf[(sdf['artist'] == 'Kanye West')]
# sad_country = gdf[(gdf['valence'] < 0.5) & (gdf['genre'] == 'country')]
# happy_hiphop = gdf[(gdf['valence'] >= 0.5) & (gdf['genre'] == 'hip hop')]

# print(sad_country.head())
# print(happy_hiphop.head())
# print(happy_hiphop.shape)
print(justin_bieber.shape)
print(justin_bieber)
# print(sad_country.shape)

(97, 6)
        Unnamed:_0      artist  \
146341    146341.0  Kanye West   
146342    146342.0  Kanye West   
146343    146343.0  Kanye West   
146344    146344.0  Kanye West   
146345    146345.0  Kanye West   
...            ...         ...   
146433    146433.0  Kanye West   
146434    146434.0  Kanye West   
146435    146435.0  Kanye West   
146436    146436.0  Kanye West   
146437    146437.0  Kanye West   

                                                      seq  \
146341  And I always find, yeah I always find somethin...   
146342  We in the streets player, get your mail\nIt's ...   
146343  Workin' this grave shift and I ain't made shit...   
146344  I'm trying to catch the beat, uh\nI'm trying t...   
146345  Oh when it all, it all falls down\nI'm telling...   
...                                                   ...   
146433  You want me to give you a testimony about my l...   
146434  No more parties in LA\nPlease baby no more par...   
146435  I'm tryna keep my faith\nW

In [ ]:
np.savetxt(r'/content/drive/MyDrive/justin.txt', justin_bieber['lyrics_token'], fmt='%s')
# np.savetxt(r'/content/drive/MyDrive/sad-country.txt', sad_country['lyrics'], fmt='%s')

In [ ]:
path_to_file = "/content/drive/MyDrive/justin.txt"
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [ ]:
vocab = sorted(set(text))
print("There are", len(vocab), "unique characters")

There are 79 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}

# And this is just an array where each ID can be the index
# for finding its corresponding char.
idx2char = np.array(vocab)

print(char2idx)
print(idx2char)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, 'a': 53, 'b': 54, 'c': 55, 'd': 56, 'e': 57, 'f': 58, 'g': 59, 'h': 60, 'i': 61, 'j': 62, 'k': 63, 'l': 64, 'm': 65, 'n': 66, 'o': 67, 'p': 68, 'q': 69, 'r': 70, 's': 71, 't': 72, 'u': 73, 'v': 74, 'w': 75, 'x': 76, 'y': 77, 'z': 78}
['\n' ' ' '!' '"' "'" '(' ')' '*' ',' '-' '.' '/' '0' '1' '2' '3' '4' '5'
 '6' '7' '8' '9' ':' ';' '?' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K'
 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' '[' ']' 'a'
 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's'
 't'

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

# Here's what the first 15 characters are
# and how they are represented with integers.
print(text[:13])
print(text_as_int[:13])

And I always 
[25 66 56  1 33  1 53 64 75 53 77 71  1]


In [ ]:
import tensorflow as tf
# Put the text into a format that tensorflow can use.
# This is our first call to one of the tensorflow functions!
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# This just looks at the first five things in the tensor stream and
# converts them to their char equivalents and prints them out.
# This is just so you can see what is in the tf data structure we're using.
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

A
n
d
 
I


In [ ]:
# seq_length is the maximum length sentence (in characters) we 
# will consider for a single input.
seq_length = 100

# Okay, let's get our sequences from the stream of characters!
# We're using the batch method from tf to do this.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# And let's just print out the first 5 so we can see.
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset

for input_example, target_example in dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

In [ ]:
# Here are the dimensions of the data structure before
# we chop it up into batches.
print(dataset)

# Batch size: number of input examples to be processed together.
BATCH_SIZE = 64

# Buffer size to shuffle the dataset.
# (F data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements.
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Here are the dimensions after organizing into batches
print(dataset)

In [ ]:
# Setting some hyperparameters.

# Length of the vocabulary in chars.
# This will be the size of the output vector, since
# we will have one probability for each possible character.
vocab_size = len(vocab)

# How many dimensions should the embeddings be?
embedding_dim = 256

# Number of units in our RNN.
rnn_units = 1024

In [ ]:
# Here's a function to construct a model.

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
# Here's a call to that function.
# Note that it's not *training* a model. It's just 
# building the model, which will be trained in the next step.

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)